In [1]:
import pandas as pd
from torch.utils.data import DataLoader
import pytorch_lightning as pl
#from pytorch_lightning.profiler import Profiler, AdvancedProfiler
import HeatfluxData as hfd
import HeatfluxModel as hfm

/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
filename_data = f'../students/bogdaale/NN-examples/scaled_QdataKn25width100microns.csv'
# Using heat flux data module
scaled_Qdata = pd.read_csv(filename_data, header=0, index_col=0)
test_set, train_set, validation_set, vis_set = hfd.heat_flux_datasets(scaled_Qdata, 0.3, 0.8)

In [3]:
### Create data loaders
train_loader = DataLoader(dataset = train_set, batch_size = 128)
validation_loader = DataLoader(dataset = validation_set, batch_size = 128)
test_loader = DataLoader(dataset = test_set, batch_size = 128)
# Special object for visualization
vis_loader = DataLoader(dataset = vis_set, batch_size = 128)

In [4]:
### Read data sacling (generated by the data manager)
data_scaling=pd.read_csv(f'../students/bogdaale/NN-examples/data_scaling.csv', header=0, index_col=0)

In [5]:
### Create the heat flux model
Nfeatures = train_set[0][0].size()[0]# TODO: find a better way than extracting the size via Tensor
print(f'Nfeatures {Nfeatures}')
Nlayer1 = 20
Nlayer2 = 10
model = hfm.DirectModel(Nfeatures, Nlayer1, Nlayer2, data_scaling)

Nfeatures 125


In [6]:
### Let's train the beast!
trainer = pl.Trainer(max_epochs = 100)
trainer.fit(model, train_loader, validation_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type    | Params
-------------------------------------
0 | fc1      | Linear  | 2.5 K 
1 | fc2      | Linear  | 210   
2 | fc3      | Linear  | 11    
3 | mse_loss | MSELoss | 0     
-------------------------------------
2.7 K     Trainable params
0         Non-trainable params
2.7 K     Total params
0.011     Total estimated model params size (MB)


/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/workspace/mlmfem/anaconda3/envs/ptl_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:  80%|████████████████████▊     | 166/208 [00:01<00:00, 157.68it/s, loss=0.054, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 1:  80%|███████████████████▉     | 166/208 [00:01<00:00, 164.71it/s, loss=0.0338, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 1:  96%|███████████████████████▉ | 199/208 [00:01<00:00, 171.64it/s, loss=0.0338, v_num=2]


Epoch 2:  80%|███████████████████▉     | 166/208 [00:01<00:00, 164.43it/s, loss=0.0187, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 3:  80%|████████████████████▊     | 166/208 [00:01<00:00, 163.67it/s, loss=0.011, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 3:  91%|███████████████████████▊  | 190/208 [00:01<00:00, 170.10it/s, loss=0.011, v_num=2]


Epoch 4:  80%|███████████████████▏    | 166/208 [00:01<00:00, 160.88it/s, loss=0.00718, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 5:  80%|███████████████████▏    | 166/208 [00:01<00:00, 164.65it/s, loss=0.00494, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 5:  87%|████████████████████▉   | 181/208 [00:01<00:00, 168.39it/s, loss=0.00494, v_num=2]


...

Epoch 96:  80%|█████████████████▌    | 166/208 [00:00<00:00, 176.13it/s, loss=7.59e-05, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 97:  80%|█████████████████▌    | 166/208 [00:00<00:00, 177.24it/s, loss=7.54e-05, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 97:  83%|██████████████████▏   | 172/208 [00:00<00:00, 178.59it/s, loss=7.54e-05, v_num=2]


Epoch 98:  80%|█████████████████▌    | 166/208 [00:00<00:00, 176.34it/s, loss=7.49e-05, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 98: 100%|██████████████████████| 208/208 [00:01<00:00, 187.53it/s, loss=7.49e-05, v_num=2]


Epoch 99:  80%|█████████████████▌    | 166/208 [00:00<00:00, 173.80it/s, loss=7.42e-05, v_num=2]
Validation: 0it [00:00, ?it/s]
Epoch 99: 100%|██████████████████████| 208/208 [00:01<00:00, 186.15it/s, loss=7.42e-05, v_num=2]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████████████████| 208/208 [00:01<00:00, 185.69it/s, loss=7.42e-05, v_num=2]
